# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing sklearn functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use pandas to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Train a simpler model with fewer words.
* Compare multiple logistic regression models.


In [2]:
import pandas as pd
import numpy as np
import string
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [3]:
products = pd.read_csv('data_files/amazon_baby.csv')
products.head(5)

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [4]:
products.shape

(183531, 3)

## Build the word count vector for each review

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

### Fill NA with blank

In [5]:
products = products.fillna({'review':''})

In [6]:
products.iloc[58,:]

name      Our Baby Girl Memory Book
review                             
rating                            5
Name: 58, dtype: object

### Remove punctuation in text

In [7]:
import string
def remove_punctuation(text):
    translator = text.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    return text

In [8]:
products['review_clean'] = products['review'].apply(remove_punctuation)
products.head(5)

,name,review,rating,review_clean
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...


## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [9]:
products = products[products['rating'] != 3]

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [10]:
def trans(x):
    if x>3:
        return +1
    else:
        return -1
products['sentiment'] = products['rating'].apply(trans)

In [11]:
products.head(3)

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1


In [12]:
products.iloc[1,:]

name                          Annas Dream Full Quilt with 2 Shams
review          Very soft and comfortable and warmer than it l...
rating                                                          5
review_clean    Very soft and comfortable and warmer than it l...
sentiment                                                       1
Name: 2, dtype: object

## Split into training and testing dataset

In [13]:
import json
with open('data_files/module-2-assignment-train-idx.json') as train_data_file:    
    train_data_idx = json.load(train_data_file)
with open('data_files/module-2-assignment-test-idx.json') as test_data_file:    
    test_data_idx = json.load(test_data_file)

print (train_data_idx[:3])
print('length of training data is:',len(train_data_idx),'\n','length of testing data is:',len(test_data_idx))

[0, 1, 2]
length of training data is: 133416 
 length of testing data is: 33336


In [14]:
train_data = products.iloc[train_data_idx]
print(train_data.shape)
train_data.head(5)

(133416, 5)


,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


In [15]:
test_data = products.iloc[test_data_idx]
print(test_data.shape)
test_data.head(5)

(33336, 5)


,name,review,rating,review_clean,sentiment
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,1
16,Nature's Lullabies First Year Sticker Calendar,"I love this little calender, you can keep trac...",5,I love this little calender you can keep track...,1
20,Nature's Lullabies Second Year Sticker Calendar,I had a hard time finding a second year calend...,5,I had a hard time finding a second year calend...,1
28,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,One of babys first and favorite books and it i...,1


## Build word count vector for each review
We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as **bag-of-word features**. Since most words occur in only a few reviews, word count vectors are **sparse**. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

- **Learn a vocabulary (set of all words) from the training data**. Only the words that show up in the training data will be considered for feature extraction.

- Compute the occurrences of the words in each review and collect them into a row vector.

- Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix **train_matrix**.

- Using the same mapping between words and columns, convert the test data into a sparse matrix **test_matrix**.

The following cell uses CountVectorizer in scikit-learn. Notice the **token_pattern** argument in the constructor.

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words and skip some unimportant words like 'it'
# First, learn vocabulary from the training data and assign columns to words, Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])
print (vectorizer.vocabulary_) # print the bag of words, number is its index of column it appears

{'it': 57486, 'came': 21721, 'early': 37328, 'and': 10505, 'was': 116798, 'not': 72510, 'disappointed': 34453, 'i': 54276, 'love': 63567, 'planet': 80500, 'wise': 119389, 'bags': 14624, 'now': 72811, 'my': 69878, 'wipe': 119288, 'holder': 52830, 'keps': 59309, 'osocozy': 75845, 'wipes': 119315, 'moist': 67820, 'does': 35380, 'leak': 60973, 'highly': 52346, 'recommend': 87458, 'very': 115480, 'soft': 97999, 'comfortable': 26721, 'warmer': 116657, 'than': 106054, 'looksfit': 63212, 'the': 106249, 'full': 45995, 'size': 96190, 'bed': 15966, 'perfectlywould': 79107, 'to': 108946, 'anyone': 11062, 'looking': 63165, 'for': 44646, 'this': 107483, 'type': 112283, 'of': 73618, 'quilt': 85937, 'is': 57196, 'a': 7280, 'product': 83729, 'well': 117906, 'worth': 120282, 'purchase': 85012, 'have': 51051, 'found': 45067, 'anything': 11108, 'else': 38507, 'like': 62056, 'positive': 81923, 'ingenious': 55742, 'approach': 11546, 'losing': 63388, 'binky': 17289, 'what': 118384, 'most': 68850, 'about': 74

In [17]:
print(type(train_matrix)) # train_matrix is a sparse matrix
print('The number of rows in train matrix is:',train_matrix.shape[0],'\n')
print('The number of words in the vocabulary retrieved from training set is:', train_matrix.shape[1])
print('The shape of the first review is:', train_matrix[0].shape)
print(train_matrix[0]) # print word count for first review, only display nonzero entries

<class 'scipy.sparse.csr.csr_matrix'>
The number of rows in train matrix is: 133416 

The number of words in the vocabulary retrieved from training set is: 121712
The shape of the first review is: (1, 121712)
  (0, 57486)	3
  (0, 21721)	1
  (0, 37328)	1
  (0, 10505)	3
  (0, 116798)	1
  (0, 72510)	2
  (0, 34453)	1
  (0, 54276)	1
  (0, 63567)	1
  (0, 80500)	1
  (0, 119389)	1
  (0, 14624)	1
  (0, 72811)	1
  (0, 69878)	2
  (0, 119288)	1
  (0, 52830)	1
  (0, 59309)	1
  (0, 75845)	1
  (0, 119315)	1
  (0, 67820)	1
  (0, 35380)	1
  (0, 60973)	1
  (0, 52346)	1
  (0, 87458)	1


In [18]:
print(test_matrix.shape)
print(test_matrix[0]) # print word count for first review in test set

(33336, 121712)
  (0, 5842)	1
  (0, 7280)	1
  (0, 9530)	1
  (0, 10440)	1
  (0, 10803)	1
  (0, 12630)	1
  (0, 13597)	2
  (0, 15930)	1
  (0, 16154)	2
  (0, 29211)	1
  (0, 37640)	1
  (0, 39997)	1
  (0, 44646)	1
  (0, 50567)	1
  (0, 50938)	1
  (0, 51051)	1
  (0, 52346)	1
  (0, 53021)	1
  (0, 53116)	1
  (0, 54276)	1
  (0, 54574)	1
  (0, 54987)	1
  (0, 55712)	1
  (0, 57196)	2
  (0, 57279)	1
  :	:
  (0, 86961)	1
  (0, 87458)	1
  (0, 87600)	1
  (0, 91035)	1
  (0, 98334)	1
  (0, 98367)	1
  (0, 100819)	1
  (0, 106102)	1
  (0, 106249)	1
  (0, 106750)	1
  (0, 107374)	1
  (0, 107483)	3
  (0, 108946)	4
  (0, 113748)	1
  (0, 116600)	1
  (0, 117337)	1
  (0, 117474)	1
  (0, 118384)	1
  (0, 118507)	2
  (0, 118577)	1
  (0, 118755)	1
  (0, 119439)	1
  (0, 120336)	1
  (0, 121347)	1
  (0, 121349)	1


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. 
- Features: This model will use the column **sparse word count matrix(train_matrix)** as features
- Target: Use the column **sentiment** of train_data as the target. 

In [19]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression(max_iter=10000)  ### set max iteration for mle
sentiment_model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(max_iter=10000)

In [20]:
# number of positive coefficients
num_positive_weights=np.sum(sentiment_model.coef_ >= 0)
print("Number of positive weights:", num_positive_weights)

Number of positive weights: 91133


In [21]:
print (sentiment_model.coef_)

[[-1.24013407e+00  3.88554677e-05  2.74944865e-02 ...  1.25408926e-02
   2.70495364e-03 -4.35668311e-05]]


## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [22]:
test_data = products.iloc[test_data_idx]

In [25]:
sample_test_data = test_data.iloc[10:13]  #observation 11,12,13
print (sample_test_data.iloc[0])

name                                    Our Baby Girl Memory Book
review          Absolutely love it and all of the Scripture in...
rating                                                          5
review_clean    Absolutely love it and all of the Scripture in...
sentiment                                                       1
Name: 59, dtype: object


### Predict scores
We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using Turi Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [26]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean']) # sample_test_matrix = word count for sample_test_data
scores = sentiment_model.decision_function(sample_test_matrix)
print (scores)

[  5.60162276  -3.16928706 -10.42900671]


### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [29]:
print (sentiment_model.classes_) # get the classes of the prediction
print ([1./(1+np.exp(-x)) for x in scores]) # calculate the probability of assigning +1 class manually
print (sentiment_model.predict_proba(sample_test_matrix)) # calculate the probability of assigning +1 class automatically

[-1  1]
[0.9963217119072274, 0.040338004926601716, 2.9561542649490867e-05]
[[3.67828809e-03 9.96321712e-01]
 [9.59661995e-01 4.03380049e-02]
 [9.99970438e-01 2.95615426e-05]]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [34]:
print(sentiment_model.predict(sample_test_matrix))

[ 1 -1 -1]


# Find the most positive (and negative) review
We now turn to examining the full test dataset, **test_data**, and use Turi Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [35]:
test_scores = sentiment_model.decision_function(test_matrix) # decision function is to print score
positive_idx = np.argsort(-test_scores)[:20]      
print (positive_idx) # index of 20 obs with most positive feedbacks
print (test_scores[positive_idx[0]]) # score of the most positive review

[18112 15732 24286 25554 24899  9125 21531 32782 30535 14482  9555 30634
 17558 26830 20743 11923  4140 30076 33060 26838]
53.80969223868255


In [36]:
negative_idx = np.argsort(test_scores)[:20]     
print(negative_idx)
print(test_scores[negative_idx[0]])

[ 2931 21700 13939  8818 28184 17069  9655 14711  1942 20594 10814  1810
  7310 31226 13751 27231 28120   205  5831 15062]
-34.714123438495356


In [38]:
predicted_result= sentiment_model.predict(test_matrix)

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

In [39]:
correct_num = np.sum(predicted_result == test_data['sentiment'])
total_num = len(test_data['sentiment'])
accuracy= correct_num / total_num
print(accuracy)

0.9322954163666907


## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [40]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']    #limit the bag of words to 20 words

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [41]:
vectorizer = CountVectorizer(vocabulary = significant_words)
train_matrix_word_subset = vectorizer.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer.transform(test_data['review_clean'])

In [43]:
train_matrix_word_subset.shape

(133416, 20)

## Train a logistic regression model on a subset of data

In [44]:
simple_model = LogisticRegression(max_iter=10000)  ### set max iteration for mle
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

LogisticRegression(max_iter=10000)

In [45]:
np.sum(simple_model.coef_ >0)

10

In [46]:
simple_model_coef_table = pd.DataFrame({'word':significant_words, 'coefficient': simple_model.coef_.flatten()})
simple_model_coef_table.sort_values(by=['coefficient'], ascending=False)

,word,coefficient
6,loves,1.673269
5,perfect,1.510263
0,love,1.363697
2,easy,1.192219
1,great,0.943950
4,little,0.520174
7,well,0.503760
8,able,0.190937
3,old,0.085424
9,car,0.058813


### Compare Models
Compare classification accuracy of sentiment model and simple model on training set

In [47]:
train_predicted_y = sentiment_model.predict(train_matrix)
correct_num = np.sum(train_predicted_y == train_data['sentiment'])
total_num = len(train_data['sentiment'])
print ("correct_num: {}, total_num: {}".format(correct_num, total_num))
train_accuracy = correct_num * 1./ total_num
print ("sentiment_model training accuracy: {}".format(train_accuracy))

train_predicted_y = simple_model.predict(train_matrix_word_subset)
correct_num = np.sum(train_predicted_y == train_data['sentiment'])
total_num = len(train_data['sentiment'])
print ("correct_num: {}, total_num: {}".format(correct_num, total_num))
train_accuracy = correct_num * 1./ total_num
print ("simple_model training accuracy: {}".format(train_accuracy))

correct_num: 129212, total_num: 133416
sentiment_model training accuracy: 0.9684895364873778
correct_num: 115648, total_num: 133416
simple_model training accuracy: 0.8668225700065959


Compare classification accuracy of sentiment model and simple model on testing set

In [48]:
test_predicted_y = sentiment_model.predict(test_matrix)
correct_num = np.sum(test_predicted_y == test_data['sentiment'])
total_num = len(test_data['sentiment'])
print ("correct_num: {}, total_num: {}".format(correct_num, total_num))
test_accuracy = correct_num * 1./ total_num
print ("sentiment_model test accuracy: {}".format(test_accuracy))

test_predicted_y = simple_model.predict(test_matrix_word_subset)
correct_num = np.sum(test_predicted_y == test_data['sentiment'])
total_num = len(test_data['sentiment'])
print ("correct_num: {}, total_num: {}".format(correct_num, total_num))
test_accuracy = correct_num * 1./ total_num
print ("simple_model test accuracy: {}".format(test_accuracy))

correct_num: 31079, total_num: 33336
sentiment_model test accuracy: 0.9322954163666907
correct_num: 28981, total_num: 33336
simple_model test accuracy: 0.8693604511639069


## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [49]:
positive_label = len(test_data[test_data['sentiment']>0])    # Equivalent to: np.sum(test_data['sentiment']>0)
print (positive_label)
negative_label = len(test_data[test_data['sentiment']<0])    # Equivalent to: np.sum(test_data['sentiment']<0)
print(negative_label)
accuracy_majority_class= positive_label/len(test_data['sentiment'])
print (accuracy_majority_class)

28095
5241
0.8427825773938085
